In [1]:
## The neuropath data at Emory ADRC is currently not particularly well organized
## and naming schema is inconsistent.  Working on developing a schema 
import os,sys, json, girder_client
import dagSecrets as dgs
from collections import Counter
import pandas as pd
import pickle
%load_ext autoreload
%autoreload 2
import brain_region_maps as brm
import adrcSchemaHelpers as hlprs
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
gc = girder_client.GirderClient(apiUrl=dgs.cbApiUrl)
gc.authenticate(apiKey=dgs.cbApiToken)

{'_id': '6102efcc7d966fafaa736885'}

In [12]:
## Load all of the item data for all WSI images
npPickleFile = 'npDemoData.pickle'
savePickle = True
reload = True

if os.path.isfile(npPickleFile) and not reload:
    with open(npPickleFile,"rb") as fp:
        npSlideDict=  pickle.load(fp)
else:
    print("RELOADING DATA")
    npSlideDict = hlprs.getADRCcollection(gc,rootCollectionName='ADRC_Data_from_APOLLO')
    if savePickle:
        with open(npPickleFile,"wb") as fp:
            pickle.dump(npSlideDict,fp)

RELOADING DATA
[{'_accessLevel': 2, '_id': '61cdda2d5aef3e5525a0e90e', '_modelType': 'collection', '_textScore': 11.0, 'created': '2021-12-30T16:11:25.674000+00:00', 'description': '', 'meta': {}, 'name': 'ADRC_Data_from_APOLLO', 'public': False, 'size': 51585345300713, 'updated': '2021-12-30T16:11:25.674000+00:00'}]


In [13]:

subjItemSet = gc.listResource('resource/%s/items?type=folder' % subj['_id'])
            if pullSlideList:
                slideList = [x for x in subjItemSet if ('svs' in x['name'] or 'ndpi' in  x['name'])]

for fldr in npSlideDict.keys():
    s,um = hlprs.scanMetadata(gc,npSlideDict[fldr],updateGirder=True)
    print(fldr,s)

1994_OS94-54 {'Processed': 67, 'girderUpdates': 0, 'slidesToUpdate': 0}
1996_E96-33 {'Processed': 118, 'girderUpdates': 0, 'slidesToUpdate': 0}
2008_E08-101 {'Processed': 92, 'girderUpdates': 0, 'slidesToUpdate': 0}
2008_E08-119 {'Processed': 25, 'girderUpdates': 0, 'slidesToUpdate': 0}
2008_E08-145 {'Processed': 46, 'girderUpdates': 0, 'slidesToUpdate': 0}
2008_E08-146 {'Processed': 15, 'girderUpdates': 0, 'slidesToUpdate': 0}
2008_E08-162 {'Processed': 172, 'girderUpdates': 0, 'slidesToUpdate': 0}
2008_E08-53 {'Processed': 68, 'girderUpdates': 0, 'slidesToUpdate': 0}
2009_E09-08 {'Processed': 90, 'girderUpdates': 0, 'slidesToUpdate': 0}
2009_E09-136 {'Processed': 89, 'girderUpdates': 0, 'slidesToUpdate': 0}
2009_E09-155 {'Processed': 96, 'girderUpdates': 0, 'slidesToUpdate': 0}
2009_E09-164 {'Processed': 269, 'girderUpdates': 0, 'slidesToUpdate': 0}
2009_E09-35 {'Processed': 12, 'girderUpdates': 0, 'slidesToUpdate': 0}
2009_E09-81 {'Processed': 127, 'girderUpdates': 0, 'slidesToUpdat

2016_E16-27 {'Processed': 75, 'girderUpdates': 70, 'slidesToUpdate': 70}
2016_E16-33 {'Processed': 63, 'girderUpdates': 62, 'slidesToUpdate': 62}
2016_E16-34 {'Processed': 58, 'girderUpdates': 56, 'slidesToUpdate': 56}
2016_E16-45 {'Processed': 73, 'girderUpdates': 68, 'slidesToUpdate': 68}
2016_E16-66 {'Processed': 113, 'girderUpdates': 100, 'slidesToUpdate': 100}
2016_OS160608 {'Processed': 86, 'girderUpdates': 0, 'slidesToUpdate': 0}
2016_OS161018 {'Processed': 66, 'girderUpdates': 0, 'slidesToUpdate': 0}
2017_E17-05 {'Processed': 102, 'girderUpdates': 96, 'slidesToUpdate': 96}
2017_E17-12 {'Processed': 115, 'girderUpdates': 110, 'slidesToUpdate': 110}
2017_E17-128 {'Processed': 58, 'girderUpdates': 56, 'slidesToUpdate': 56}
2017_E17-129 {'Processed': 48, 'girderUpdates': 46, 'slidesToUpdate': 46}
2017_E17-138 {'Processed': 78, 'girderUpdates': 72, 'slidesToUpdate': 72}
2017_E17-140 {'Processed': 56, 'girderUpdates': 55, 'slidesToUpdate': 55}
2017_E17-141 {'Processed': 76, 'girderUp

In [ ]:
# hlprs.unmatchedFileNames

In [ ]:
## meta.region  meta.stain  meta.subject
basicSlideInfo = []
mdKeys = ['region','stain','subject','brain_region']

for subjyr in npSlideDict:
    for s in npSlideDict[subjyr]:
        slideData = {'folderId': subjyr,'slideName':s['name'],'itemId':s['_id']}
        
        for k in mdKeys:
            if k in s['meta']:
                slideData[k] = s['meta'][k]
        basicSlideInfo.append(slideData)
pd.DataFrame(basicSlideInfo).to_csv("adrcInitialMetadataExport.csv")

In [ ]:
npByYearCase = {}
for k in npSlideDict:
    try:
        (year,caseId) = k.split("_")
        if year not in npByYearCase:
            npByYearCase[year]= []
            
        npByYearCase[year].append(caseId)
        
    except:
        print("Weird pattern for",k)

In [ ]:
# from ipywidgets import Dropdown, interactive, HBox, interactive_output
# from collections import Counter
# npYearList = list(npByYearCase.keys())

# dpYear = Dropdown(options=npYearList,description='Year')
# dpCaseId = Dropdown(options=npByYearCase[npYearList[0]],description='CaseId')


# def on_change(*args):
#     value = dpYear.value
#     dpCaseId.options = npByYearCase[value]
# dpYear.observe(on_change, names='value')

# def analyzeNpCase(year, caseId):
#     print("Gutman needs a bath")
#     ### This will perform an analysis of a folder of images to see the number of slides 
#     ## And check the metadata
#     npKey = "%s_%s" % ( year,caseId)
#     slideList = npSlideDict[npKey]
    
#     ## Keys of interest
#     koi = ['region','regionId','subject','stain','Stain','Region']
#     slideKeys = {}
#     for k in koi:
#         slideKeys[k]= []
    
#     caseMeta = [] ## 
#     d=caseMeta ## Exposing this as a global
    
#     for s in slideList:
#         meta = s['meta']
#         for k in meta:
#             if k in koi:
#                 slideKeys[k].append(meta[k])
#         caseMeta.append( hlprs.validateSlide(s,caseId) )
                
                
#     ## Now go through and make things into counters so it's easier to see
#     for k in koi:
#         slideKeys[k] = Counter(slideKeys[k])
                
#     print("Unique RegionId Count: %d \t Distinct SubjID: \t%d"% (len(slideKeys['region']),len(slideKeys['subject'])))
#     print("Total slides found: %d" % len(slideList))  ## Will add in schema maps as well
#     df = pd.DataFrame(caseMeta)
# #     print(df)
#     if 'region' in df.columns:
#         print(df.region.value_counts())
#     else:
#         print("region tag not found..")
#         print([s['name'] for s in slideList])

    
# ui = HBox([dpYear,dpCaseId])
# w = interactive_output(analyzeNpCase, {'year':dpYear, 'caseId':dpCaseId})
# display(ui,w)

# ## Key I am interested in is npSchema going forward
# ## Need to figure out what to do with control slides..  will add a key to the Schema that's optional called controlSlide


In [2]:
# ## LEt's examine a specific case...  2020_E20-106
# import re

# caseId = '2020_E20-106'
# npSlideDict[caseId]

# adrcName_re = re.compile('(?P<subject>.\d\d-\d+)_(?P<blockID>\d+) (?P<stain>.*)\.[svs|ndpi]')

# ## Let me look at the filenames in this dictionary
# for slideName in [x['name'] for x in npSlideDict[caseId]]:
#     m =  adrcName_re.search(slideName)
#     if m:
#         print(slideName,m.groupdict())
#     elif ('CON' in slideName.upper() ):  ### This is probably a control slide
#         print("Control slide?",slideName)
#     else:
#         print("Shit nothing works",slideName)